# Categorical Data Type

This chapter covers the categorical data type which is unique to pandas and does not exist within numpy. The categorical data type is often used whenever a column of data has known, limited, discrete values that repeat. This is often the case for string columns and is easiest to understand with an example. Let's read in the City of Houston employee dataset.

In [ ]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

## Changing data types to categorical

Any column may be converted to the categorical data type, but most often it is string and occasionally integer columns that are chosen to be categorical. In this dataset, each string column is a good candidate to be categorical. Let's select the `dept` column and count the occurrences of each unique value.

In [ ]:
dept = emp['dept']
dept.value_counts()

### Values are known, limited, discrete, and repeat

In this section, we discuss the properties that make a particular column a good candidate to convert to categorical. The total possible number of values in the column should be **known**. There shouldn't be any mysterious values that will appear when future data of the same kind is collected. With our department column, it's likely that we will be aware of each individual department and that new departments will not be created often.

The unique values in the column should be **limited** and much less than the total number of values. There are only 9 unique department values which is substantially less than the total number of values (24,000+). 

The values should be **discrete**, meaning that there are no partial values. Each value in the column must be one of the known categories. With our data, each employee works in exactly one department. There is no partial department. You either work in one department or another, but not both. You must be one of those 9 categories. 

Finally, the values must **repeat**, and they will repeat if the first three properties hold for your data. As you can see from the output of the `value_counts` methods, each value repeats many times. Due to all these properties, the `dept` can be and should be converted to the categorical data type. 

### Converting to categorical with the `astype` method

The simplest way to convert a Series to categorical is to pass the string 'category' to the `astype` method. We assign this new Series to the `dept_cat` variable name.

In [ ]:
dept_cat = dept.astype('category')
dept_cat.head()

### Visual display of a categorical Series

The output looks a bit different than a normal Series. Let's verify that we do indeed have a Series.

In [ ]:
type(dept_cat)

The index and the values of our new categorical Series appear identical to the output of a Series that has the object data type. The difference is the appearance of the unique categories below the Series.

### The formal data type

Let's verify that the data type is categorical by accessing the `dtype` attribute.

In [ ]:
dept_cat.dtype

### What is CategoricalDtype?

The formal data type name for categorical data is `CategoricalDtype`. This can be confusing since pandas also uses the term 'category' in the Series output.

### Is the data type category, categorical or CategoricalDtype?

With the output above, we have three different terms for the same data type - category, categorical, and CategoricalDtype. I'm not sure what to say here - pandas has several inconsistencies and the terminology surrounding categorical data is one of them.

The formal data type is `CategoricalDtype` but more informally it will be referred to as 'categorical'. This is similar to how the numbers 1, 5, and 10 are informally referred to as 'integers' but have a formal data type of 'int64'.

## Why the categorical data type is useful

On the surface, our categorical Series looks very similar to its object Series counterpart, but there are some significant differences.

### Internal storage of categorical data

Categorical data is stored much more efficiently than object data containing strings. Each unique value in a column of categorical data is stored **once** regardless of how many times it repeats in the Series. Also, each of the unique values has an integer code that refers to it. It is these integers that are stored in memory to represent the data. 

Object columns store each value in a unique location in memory. For instance, the string 'Police' appears over 7,000 times in the departments Series. Each one of these strings is stored in a unique location in memory. Using integers to represent categories can save a tremendous amount of memory.

### Example of categorical storage

Let's create a simplified example to show how pandas stores categorical data internally using Python lists. In this example, we'll have three unique departments. They are stored exactly once in the list `cats` below. The actual data is contained in the `vals` list containing the values 0, 1, or 2. 

In [ ]:
cats = ['Police', 'Fire', 'Library']
vals = [1, 1, 0, 2, 0, 1, 2, 2, 1, 2, 1]

Each integer corresponds to the category located at that index location in the `cats` list. We can convert each value in the `vals` list to its corresponding category using the list comprehension below. 

In [ ]:
[cats[val] for val in vals]

pandas stores its categorical data similarly to the example. We can access the internal storage mechanisms for categorical data with the `cat` accessor.

## The `cat` accessor

We previously covered the `str` and `dt` accessors which provide us special access to string-only and datetime-only attributes and methods. The `cat` accessor provides us with more special attributes and methods for categorical Series. Let's take a look at some of the important attributes and methods it provides.

### Get the categories

The unique sequence of categories can be retrieved with the `categories` attribute.

In [ ]:
dept_cat.cat.categories

### Get the integer codes

The underlying integer codes may be retrieved with the `codes` attribute which returns a Series the same length as the original. Notice that it uses 8-bit integers to store the data.

In [ ]:
dept_cat.cat.codes.head()

### Verify codes correspond with categories

The first three values of the `dept_cat` Series are 'Police', 'Other', and 'Houston Public Works'. Let's verify that the first three integers of the `codes` attribute from above are 7, 5, and 3. Let's verify that these codes correspond with the categories.

In [ ]:
dept_cat.cat.categories[7]

In [ ]:
dept_cat.cat.categories[5]

In [ ]:
dept_cat.cat.categories[3]

## Massive reduction in memory used

One of the biggest benefits of using categorical columns is the amount of memory saved. Instead of using a string for every value, an integer code is used. Integers take up significantly less space than strings. pandas also uses the smallest integer size to store the codes. For instance, if there are less than 128 categories, an int8 is used.

### The memory_usage method

pandas provides the `memory_usage` method to return the number of bytes used by the Series. To get the exact memory for string columns you need to set the parameter `deep` to `True`. Let's get the original amount of memory used.

In [ ]:
orig_mem = dept.memory_usage(deep=True)
orig_mem

Use the method again to get the memory used on our categorical Series.

In [ ]:
cat_mem = dept_cat.memory_usage(deep=True)
cat_mem

Let's find the percentage reduction in memory used by converting to categorical.

In [ ]:
1 - cat_mem / orig_mem

An astounding 98.5% reduction in memory takes place. Using 8-bit integers instead of the entire string made a huge difference.

## Speeding up operations

Another nice benefit of using the categorical data type is the increase in performance for most operations. It's much faster to use integer arrays instead of object arrays. Let's cover a few examples that show this performance improvement.

### Comparison operators

The comparison operators should complete much faster. In the example below, we are testing equality of each value to the string 'Police'. Using the categorical Series results in nearly 10 times better performance.

In [ ]:
%timeit -n 5 dept == 'Police'

In [ ]:
%timeit -n 5 dept_cat == 'Police'

### Sorting

Sorting also executes much faster.

In [ ]:
%timeit -n 5 dept.sort_values()

In [ ]:
%timeit -n 5 dept_cat.sort_values()

### Most other operations

Most other operations happen faster. Here, we see the difference when using the `value_counts` method.

In [ ]:
%timeit -n 5 dept.value_counts()

In [ ]:
%timeit -n 5 dept_cat.value_counts()

## The str accessor is still available

Even though we've converted to the categorical data type, the `str` accessor is still available to use as long as the original data contained strings. Here, we make all the values uppercase.

In [ ]:
dept_cat.str.upper().head()

Unfortunately, an object Series is returned for all the `str` accessor methods that return strings so you'll have to convert it again to categorical after the operation completes if you want to keep it as a categorical.

## Ordered categories

Generally speaking, there are two types of categorical data, **nominal** and **ordinal**. Nominal categorical data is when the values have no natural ordering. When the values do have a natural ordering, they are referred to as **ordinal** data. To help remember, both ordinal and order begin with 'ord'.

The departments from above are a good example of nominal data as no department has any natural precedence over the other. There is no good example in our employee dataset of ordinal data. Let's read in the diamonds dataset which has information on the size, cut, color, clarity, and price for many diamonds.

In [ ]:
diamonds = pd.read_csv('../data/diamonds.csv')
diamonds.head()

In [ ]:
diamonds.shape

If you are familiar with diamonds, then you know that cut, color, and clarity have a specific ordering corresponding to the quality of that property. Let's take a look at the clarity column's unique values.

In [ ]:
clarity = diamonds['clarity']
clarity.value_counts()

Before creating an ordered categorical, let's convert clarity to an unordered categorical like we did above. Passing the `astype` method the string 'category' always creates an unordered categorical data type.

In [ ]:
clarity_cat = diamonds['clarity'].astype('category')
clarity_cat.head()

We can use the `ordered` attribute available from the `cat` accessor to verify that it is unordered.

In [ ]:
clarity_cat.cat.ordered

The data dictionary contains information on the ordering of each categorical variable. Let's read it, changing the column display width option so that we can read all of the description for clarity.

In [ ]:
pd.set_option('display.max_colwidth', 100)
pd.read_csv('../data/dictionaries/diamonds_dictionary.csv')

### Creating an ordered categorical

Creating an ordered categorical column takes a bit of work. Here are the three steps that you need to take:

1. Create a list of all of the unique categories in the order that you desire
1. Use the `CategoricalDtype` constructor available directly from `pd`. Pass it the list of categories and set the `ordered` parameter to `True`. Assign this result to a variable name.
1. Pass the variable name in step 2 to the `astype` Series method

Here, we complete the three steps and assign the new ordered categorical Series to `clarity_ordered_cat`.

In [ ]:
cats = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
clarity_dtype = pd.CategoricalDtype(cats, ordered=True)
clarity_ordered_cat = clarity.astype(clarity_dtype)
clarity_ordered_cat.head()

Notice that the categories appear at the bottom with less than signs separating them indicating the order. Let's verify the data type of our new Series.

In [ ]:
clarity_ordered_cat.dtype

Let's also verify that it is ordered.

In [ ]:
clarity_ordered_cat.cat.ordered

## Special properties of ordered categorical Series

Ordered categorical Series have some behavior that differs from their unordered counterparts. For instance, taking the maximum or minimum value returns the category ranked as the best or worst.

In [ ]:
clarity_ordered_cat.max()

Attempting to call the `max` method on an unordered categorical raises an error.

In [ ]:
clarity_cat.max()

The original object Series does work with the `max` method and returns the value with greatest alphabetical value which is different than the best measurement for clarity.

In [ ]:
clarity.max()

Sorting ordered categoricals is done by their category order and not alphabetically. Here we call the `value_counts` method on both the ordered and unordered clarity Series and then sort the index.

In [ ]:
clarity_ordered_cat.value_counts().sort_index()

Sorting the unordered categorical index sorts alphabetically.

In [ ]:
clarity_cat.value_counts().sort_index()

## Integers can be categories

As mentioned, any column regardless of its data type may be converted to categorical and not just strings. If the values are known, limited, discrete, and repeat then they are good candidates for categorical data. Integers are the primary non-string data type that can represent categorical data. Here are some examples of integer categorical data:

* Rating of a movie/hotel/restaurant given that the range is known such as the integers one through five
* Zip codes for a particular city
* Numeric error codes

As with our string columns, integer categorical columns may be unordered (like zip codes) or ordered (like movie ratings). Let's read in just three columns of the housing dataset.

In [ ]:
housing = pd.read_csv('../data/housing.csv', 
                      usecols=['MSSubClass', 'OverallQual', 'SalePrice'])
housing.head()

By default, these columns are read in as integers. Looking at the data dictionary, both 'MSSubClass' and 'OverallQual' are categoricals. The 'MSSubClass' identifies the type of dwelling. There is no inherent order for these values. Here are a few of its integer codes and their corresponding description.

* 20 - 1-STORY 1946 & NEWER ALL STYLES
* 30 - 1-STORY 1945 & OLDER
* 40 - 1-STORY W/FINISHED ATTIC ALL AGES
* 45 - 1-1/2 STORY - UNFINISHED ALL AGES
* 50 - 1-1/2 STORY FINISHED ALL AGES

The data dictionary also informs us that the 'OverallQual' column contains integer values 1 through 10 with higher numbers indicating a higher overall material and finish of the house. The values in this column have a clear ordinal

### Converting multiple columns to categorical

The `astype` method may be used to convert multiple columns to different data types. This is done using a dictionary mapping the column name to the new data type. Let's use the string 'category' to convert MSSubClass and `pd.CategoricalDtype` to create an ordered categorical data type for OverallQual. The `astype` method produces a new copy of the data which forces us to assign its result to a new variable name.

In [ ]:
oq_cats = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
oq_dtype = pd.CategoricalDtype(oq_cats, ordered=True)
housing_new = housing.astype({'MSSubClass': 'category', 'OverallQual': oq_dtype})
housing_new.head()

Let's verify that that the data types have changed.

In [ ]:
housing_new.dtypes

The `dtypes` attribute does not distinguish between unordered and ordered categorical data. Use the `dtype` Series attribute to get the exact type of categorical data.

In [ ]:
housing_new['MSSubClass'].dtype

In [ ]:
housing_new['OverallQual'].dtype

## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">Create a new DataFrame `diamonds2` that has the columns clarity, color, and cut as ordered categoricals.</span>

### Exercise 2

<span  style="color:green; font-size:16px">Find the number of occurrences for each cut type sorted by its cut order.</span>